In [1]:
# Importing dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Importing and reading the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Dropping the non-beneficial ID columns, 'EIN' and 'NAME'.
application_clean = application_df.drop(columns=['EIN', 'NAME'])
application_clean.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
# application_clean.nunique()

# Look at APPLICATION_TYPE value counts for binning
# application_clean['APPLICATION_TYPE'].value_counts()

In [4]:
# Choosing a cutoff value and creating a list of application types to be replaced

application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_clean['APPLICATION_TYPE'] = application_clean['APPLICATION_TYPE'].replace(app,"Other")

# Checking to make sure binning was successful
application_clean['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Look at CLASSIFICATION value counts for binning
# application_clean['CLASSIFICATION'].value_counts()

# You may find it helpful to look at CLASSIFICATION value counts >1
# vc = application_clean['CLASSIFICATION'].value_counts()
# print (vc[vc>1])

In [6]:
# Choosing a cutoff value and create a list of classifications to be replaced

classifications_to_keep = ['C1000', 'C2000', 'C1200', 'C3000', 'C2100']

application_clean.loc[~application_clean['CLASSIFICATION'].isin(classifications_to_keep), 'CLASSIFICATION'] = 'Other'


# Checking to make sure binning was successful
application_clean['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [7]:
# Converting categorical data to numeric with `pd.get_dummies`
classification_dummies = pd.get_dummies(application_clean)
classification_dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [8]:
#  checking to see if the sample needs to be adjusted
classification_dummies['IS_SUCCESSFUL'].value_counts()

1    18261
0    16038
Name: IS_SUCCESSFUL, dtype: int64

In [9]:
classification_dummies.columns

Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_Other',
       'APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3',
       'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T8',
       'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200',
       'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C3000',
       'CLASSIFICATION_Other', 'USE_CASE_CommunityServ', 'USE_CASE_Heathcare',
       'USE_CASE_Other', 'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Co-operative',
       'ORGANIZATION_Corporation', 'ORGANIZATION_Trust', 'INCOME_AMT_0',
       'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M

In [10]:
# Splitting the preprocessed data into features and target arrays

# dropping the "Special Consiterations" columns
X = classification_dummies.drop(["IS_SUCCESSFUL"], axis='columns').values
y = classification_dummies["IS_SUCCESSFUL"].values

# Splitting the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [11]:
# Creating a StandardScaler instances
scaler = StandardScaler()

# Fitting the StandardScaler
X_scaler = scaler.fit(X_train)

# Scaling the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## First Revision

In [12]:
# Defining the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# adding a third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Checking the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               4400      
                                                                 
 dense_1 (Dense)             (None, 30)                3030      
                                                                 
 dense_2 (Dense)             (None, 10)                310       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 7,751
Trainable params: 7,751
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compileing the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Training the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 6s 5ms/step - loss: 0.5741 - accuracy: 0.7194
Epoch 2/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5556 - accuracy: 0.7275
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5528 - accuracy: 0.7296
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5506 - accuracy: 0.7315
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5493 - accuracy: 0.7326
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5483 - accuracy: 0.7319
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5484 - accuracy: 0.7333
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5470 - accuracy: 0.7343
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5464 - accuracy: 0.7330
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5459 - accura

In [15]:
# Evaluating the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5592 - accuracy: 0.7291 - 502ms/epoch - 2ms/step
Loss: 0.5591995716094971, Accuracy: 0.7290962338447571


## Second Revision

In [16]:
# Defining the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  200
hidden_nodes_layer2 = 100
hidden_nodes_layer3 = 50

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# adding a third hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Checking the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 200)               8800      
                                                                 
 dense_5 (Dense)             (None, 100)               20100     
                                                                 
 dense_6 (Dense)             (None, 50)                5050      
                                                                 
 dense_7 (Dense)             (None, 1)                 51        
                                                                 
Total params: 34,001
Trainable params: 34,001
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compileing the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
# Training the model
fit_model = nn2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5667 - accuracy: 0.7225
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5548 - accuracy: 0.7292
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5517 - accuracy: 0.7308
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5505 - accuracy: 0.7305
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5492 - accuracy: 0.7326
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5490 - accuracy: 0.7327
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5472 - accuracy: 0.7342
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5472 - accuracy: 0.7341
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5467 - accuracy: 0.7342
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5458 - accura

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5752 - accuracy: 0.7282 - 478ms/epoch - 2ms/step
Loss: 0.5752055048942566, Accuracy: 0.7281632423400879


# Third Revision

In [20]:
# Defining the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  200
hidden_nodes_layer2 = 100
# hidden_nodes_layer3 = 50

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# adding a third hidden layer
# nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Checking the structure of the model
nn3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 200)               8800      
                                                                 
 dense_9 (Dense)             (None, 100)               20100     
                                                                 
 dense_10 (Dense)            (None, 1)                 101       
                                                                 
Total params: 29,001
Trainable params: 29,001
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Compileing the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [22]:
# Training the model
fit_model = nn3.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 3s 2ms/step - loss: 0.5706 - accuracy: 0.7184
Epoch 2/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5567 - accuracy: 0.7276
Epoch 3/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5543 - accuracy: 0.7271
Epoch 4/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5527 - accuracy: 0.7299
Epoch 5/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5505 - accuracy: 0.7312
Epoch 6/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5500 - accuracy: 0.7314
Epoch 7/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5498 - accuracy: 0.7294
Epoch 8/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5486 - accuracy: 0.7309
Epoch 9/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5485 - accuracy: 0.7306
Epoch 10/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5476 - accura

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5662 - accuracy: 0.7317 - 1s/epoch - 4ms/step
Loss: 0.5661851763725281, Accuracy: 0.7316617965698242


In [25]:
nn3.save("AlphabetSoupCharity_Optimization.h5")